In [ ]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords as s
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

import re
import time

## **LOAD DATA AND SET PARAMETERS**

In [ ]:
'''
LOAD DATA AND SET PARAMETERS
Load training and combined validation-test datasets from excel
Note this was previously split to allow data augmentation of training data only
set_aug: True will load augmented dataset, False non-augmented
set_label = 'Protocol' or 'UrgBin'
dir_path: path to directory containing data, please change to your own directory
'''
set_aug = True
set_label = 'Protocol'
dir_path = "/content/drive/MyDrive/Colab Notebooks/Final Notebooks/Submission/"

if set_aug:
    X_train = pd.read_csv(dir_path + "Data/Xtrain_aug_1000-400.csv")['Clinical history'].values.tolist()
    X_test = pd.read_csv(dir_path + "Data/Xtest_aug_1000-400.csv")['Clinical history'].values.tolist()

    y_train = pd.read_csv(dir_path + "Data/ytrain_aug_1000-400.csv")[set_label].values.tolist()
    y_test = pd.read_csv(dir_path + "Data/ytest_aug_1000-400.csv")[set_label].values.tolist()

else:
    X_train = pd.read_csv(dir_path + "Data/Xtrain_70-30_no-aug.csv")['Clinical history'].values.tolist()
    X_test = pd.read_csv(dir_path + "Data/Xtest_70-30_no-aug.csv")['Clinical history'].values.tolist()

    y_train = pd.read_csv(dir_path + "Data/ytrain_70-30_no-aug.csv")[set_label].values.tolist()
    y_test = pd.read_csv(dir_path + "Data/ytest_70-30_no-aug.csv")[set_label].values.tolist()


In [ ]:
def clean_text(data):

    cleaned_text = []
    for i in data:
        text = i.lower()
        text = re.sub("[^a-zA-Z0-9]", " ", text)
        text = re.sub("(?<!\S)\d+(?!\S)", " ", text)
        text = nltk.word_tokenize(text)
        text = [wnl.lemmatize(word) for word in text]
        text = [word for word in text if word not in stop_words]
        text  = " ".join(text)
        cleaned_text.append(text)

    return cleaned_text


In [ ]:
# DATA PREPROCESSING

processed_text = []
wnl = WordNetLemmatizer()
stop_words = s.words("english")

X_train = clean_text(X_train)
X_test = clean_text(X_test)

print(len(X_train))
print(len(X_test))

In [ ]:
'''
GRIDSEARCH - this section is commented out, previously used to find best parameters for SVM
'''
'''
# Using GridSearch SVC to find best parameters for SVM.

param_linear = [
		 {"kernel":
		["linear"],
		"C": [0.001, 0.1, 1, 10, 100]}]

param_poly = [
		 {"kernel":
		["poly"],
		"degree": [2, 3, 4],
		"C": [0.001, 0.1, 1, 10, 100]}]

param_rbf = [
		 {"kernel":
		["rbf"],
		"gamma": ["auto", "scale"],
		"C": [0.001, 0.1, 1, 10, 100]}]


vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

grid_linear = GridSearchCV(SVC(),param_linear,refit = True, verbose=2)
grid_poly = GridSearchCV(SVC(),param_poly,refit = True, verbose=2)
grid_rbf = GridSearchCV(SVC(),param_rbf,refit = True, verbose=2)

grid_linear.fit(X_train_tfidf, y_train)
print("Linear:")
print(grid_linear.best_params_)

grid_poly.fit(X_train_tfidf, y_train)
print("Polynomial:")
print(grid_poly.best_params_)

grid_rbf.fit(X_train_tfidf, y_train)
print("RBF:")
print(grid_rbf.best_params_)
'''


In [ ]:
model = SVC(kernel='linear', C=1)
clf = Pipeline([('tfidf', TfidfVectorizer()),
                      ('model', model)])
start = time.time()
clf.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {stop - start}s")
y_pred = clf.predict(X_test)
classification = classification_report(y_test, y_pred)

print(classification)

print(f"Training time: {stop - start}s")